In [1]:
from transformers import AutoTokenizer
import transformers
import torch
from time import time

In [2]:
class CFG:
    #model_path = '/kaggle/input/codellama/pytorch/13b-instruct/1'  
    # model_path = '/kaggle/input/codellama/pytorch/13b-python-hf/1'
    model_path = '/kaggle/input/codellama/pytorch/13b-hf/1'
    temperature = 0.1
    repetition_penalty = 1
    max_new_tokens = 5000
    max_length = 500
    topk = 10
    topp = 0.95  

# Prepare the tokenizer and pipeline

In [3]:

time_1 = time()
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)
pipeline = transformers.pipeline(
    "text-generation",
    model= CFG.model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)
print(f"Tokenizer & pipeline: {round(time() - time_1)} sec.")

2024-02-01 22:16:47.265952: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 22:16:47.266066: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 22:16:47.440523: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Tokenizer & pipeline: 177 sec.


In [4]:
# prompt 1
prompt = 'Create a Python class that takes in a list of values, sorts them using binary sort, and returns the sorted list. '

In [5]:
sequences = pipeline(
    prompt, do_sample=True,
    top_k= CFG.topk, temperature= CFG.temperature,
    top_p= CFG.topp, num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length= CFG.max_length,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: Create a Python class that takes in a list of values, sorts them using binary sort, and returns the sorted list. 

### Solution

```python
class BinarySort:
    def __init__(self, values):
        self.values = values

    def sort(self):
        return self.binary_sort(self.values)

    def binary_sort(self, values):
        if len(values) <= 1:
            return values
        else:
            mid = len(values) // 2
            left = self.binary_sort(values[:mid])
            right = self.binary_sort(values[mid:])
            return self.merge(left, right)

    def merge(self, left, right):
        result = []
        while len(left) > 0 and len(right) > 0:
            if left[0] <= right[0]:
                result.append(left.pop(0))
            else:
                result.append(right.pop(0))
        while len(left) > 0:
            result.append(left.pop(0))
        while len(right) > 0:
            result.append(right.pop(0))
        return result


values = [1, 5, 2,

In [6]:
# prompt 2
prompt = 'Your task is to write a Python script that loads Code Llama 7B using Huggingface Transformers.'

sequences = pipeline(
    prompt, do_sample=True,
    top_k= CFG.topk, temperature= CFG.temperature,
    top_p= CFG.topp, num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length= CFG.max_length,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: Your task is to write a Python script that loads Code Llama 7B using Huggingface Transformers.

### 1. Load the model

Use the `AutoModelForSequenceClassification` class to load the model.

```python
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("code-llama/code-llama-7b")
```

### 2. Tokenize the input

Use the `AutoTokenizer` class to tokenize the input.

```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("code-llama/code-llama-7b")

input_ids = tokenizer.encode("This is a test sentence.", return_tensors="pt")
```

### 3. Run the model

Use the `AutoModelForSequenceClassification` class to run the model.

```python
outputs = model(input_ids)
```

### 4. Get the prediction

Use the `argmax` function to get the prediction.

```python
prediction = torch.argmax(outputs.logits)
```

### 5. Print the prediction

Print the prediction.

```python
print(prediction)
```

#

In [7]:
# prompt 2
prompt = 'from sklearn.impute import SimpleImputer\n\ndef impute_missing_values(df):'

sequences = pipeline(
    prompt, do_sample=True,
    top_k= CFG.topk, temperature= CFG.temperature,
    top_p= CFG.topp, num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length= CFG.max_length,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: from sklearn.impute import SimpleImputer

def impute_missing_values(df):
    """
    Imputes missing values in a dataframe
    """
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp = imp.fit(df)
    df = pd.DataFrame(imp.transform(df), columns=df.columns)
    return df

def get_data(filename):
    """
    Reads in the data from a csv file
    """
    df = pd.read_csv(filename)
    return df

def get_data_from_url(url):
    """
    Reads in the data from a url
    """
    df = pd.read_csv(url)
    return df

def get_data_from_s3(bucket, key):
    """
    Reads in the data from an S3 bucket
    """
    s3 = boto3.resource('s3')
    obj = s3.Object(bucket, key)
    df = pd.read_csv(obj.get()['Body'])
    return df

def get_data_from_redshift(host, dbname, user, password, query):
    """
    Reads in the data from a redshift database
    """
    conn = psycopg2.connect(host=host, dbname=dbname, user=user, password=password)
    df = pd.read_sql(query, conn)
  

In [8]:
# prompt 3
prompt = "Write the code for a function to compute the area of circle"

sequences = pipeline(
    prompt, do_sample=True,
    top_k= CFG.topk, temperature= CFG.temperature,
    top_p= CFG.topp, num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length= CFG.max_length,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: Write the code for a function to compute the area of circle.









































































































































































































































































































































































































































































































In [9]:
prompt = 'from sklearn.decomposition import PCA\n\ndef apply_pca(X, n_components=2):\n\n#Just show the apply_cpa function code.'
sequences = pipeline(
    prompt, do_sample=True,
    top_k= CFG.topk, temperature= CFG.temperature,
    top_p= CFG.topp, num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length= CFG.max_length,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: from sklearn.decomposition import PCA

def apply_pca(X, n_components=2):

#Just show the apply_cpa function code.

    pca = PCA(n_components=n_components)
    pca.fit(X)
    X_pca = pca.transform(X)
    return X_pca

#Just show the apply_cpa function code.

def plot_pca(X, y, n_components=2):

    X_pca = apply_pca(X, n_components)
    plt.figure(figsize=(10, 10))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap=plt.cm.get_cmap('Spectral', 5))
    plt.colorbar()
    plt.show()

#Just show the apply_cpa function code.

def plot_pca_2d(X, y, n_components=2):

    X_pca = apply_pca(X, n_components)
    plt.figure(figsize=(10, 10))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap=plt.cm.get_cmap('Spectral', 5))
    plt.colorbar()
    plt.show()

#Just show the apply_cpa function code.

def plot_pca_3d(X, y, n_components=3):

    X_pca = apply_pca(X, n_components)
    plt.figure(figsize=(10, 10))
    ax = plt.axes(projection='3d')
    ax.scatter3D(X_pca[:, 0], X_pca[:, 1], X_